In [1]:
#!pip install --upgrade pip --user
#!pip install configparser --user
import os, sys
import configparser
sys.path.append('/home/jovyan/.local/lib/python3.6/site-packages/')
print(sys.path)

os.path.abspath("AzureDownload/config.txt")
os.getcwd()
config = configparser.ConfigParser()
config.read("/home/jovyan/AzureDownload/config.txt")
config.sections()

['/home/jovyan/ReproducibilityDemo/code-data', '/srv/conda/lib/python37.zip', '/srv/conda/lib/python3.7', '/srv/conda/lib/python3.7/lib-dynload', '', '/home/jovyan/.local/lib/python3.7/site-packages', '/srv/conda/lib/python3.7/site-packages', '/srv/conda/lib/python3.7/site-packages/IPython/extensions', '/home/jovyan/.ipython', '/home/jovyan/.local/lib/python3.6/site-packages/']


['configuration', 'credentials']

In [2]:
## Required for Azure Data Lake Analytics job management
#!pip install azure-mgmt-datalake-analytics --user
from azure.mgmt.datalake.analytics.job import DataLakeAnalyticsJobManagementClient
from azure.mgmt.datalake.analytics.job.models import JobInformation, JobState, USqlJobProperties
from azure.common.credentials import ServicePrincipalCredentials

credentials = ServicePrincipalCredentials(client_id =config.get("credentials","client"), secret=config.get("credentials","password"), tenant =config.get("credentials","tenant"))

subid= 'Microsoft Azure Sponsorship'
rg = 'IUNI-MAG-RG'
location = 'eastus2'
adla = 'iunimag'

adlaJobClient = DataLakeAnalyticsJobManagementClient(credentials, 'azuredatalakeanalytics.net')

### After seting up the environement and Azure crendentials, we submit a u-sql script to run on Azure Datalake Analytics 

In [3]:
import logging, getpass, pprint, uuid, time
script = open("/home/jovyan/openScience/code-data/usql-scripts/MAGgetOld.usql", "r")

jobId = str(uuid.uuid4())
jobResult = adlaJobClient.job.create(
    adla,
    jobId,
    JobInformation(
        name='MAGgetOld@jupyter',
        type='USql',
        degree_of_parallelism=15,
        properties=USqlJobProperties(script=script.read() )
    )
)

In [3]:
import time
from azure.storage.blob import BlockBlobService

CONTAINERNAME ='mag-2019-03-02'
BLOBNAME= "Open-old.csv"
LOCALFILENAME= "/home/jovyan/openScience/code-data/Open-old.csv"

#download from blob
block_blob_service=BlockBlobService(account_name=config.get("configuration","account"),account_key=config.get("configuration","password"))
#download from blob
t1=time.time()
block_blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)
t2=time.time()
print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

It takes 1.569260597229004 seconds to download Open-old.csv


In [4]:
BLOBNAME= "Reproduce-old.csv"
LOCALFILENAME= "/home/jovyan/openScience/code-data/Reproduce-old.csv"

#download from blob
t1=time.time()
block_blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)
t2=time.time()
print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

It takes 0.9228744506835938 seconds to download Reproduce-old.csv


### With the output tables downloaded, we proceed to parse the inverted-indexed abstracts

In [3]:
import pandas as pd

def parse(inputStr):
  if pd.isna(inputStr):
    return None
  s = eval(inputStr)
  List = list()
  for word in s['InvertedIndex'].keys():
    for index in s['InvertedIndex'][word]:
        List.append((word, index))
  List.sort(key=lambda x: x[1], reverse=False)
  article = [i[0] for i in List]
  article = ' '.join(article)
  return article
  pass

#eval(open.iloc[3]['IndexedAbstract'])['InvertedIndex']['Research']

In [7]:
open = pd.read_csv('Open-old.csv', delimiter="\t")
open = open[open['AuthorIds'].notnull()]
open['IndexedAbstract'] = open['IndexedAbstract'].map(parse)

rep = pd.read_csv('Reproduce-old.csv', delimiter="\t")
rep = rep[rep['AuthorIds'].notnull()]
rep['IndexedAbstract'] = rep['IndexedAbstract'].map(parse)
#rep.count()

AuthorIds               7907
AuthorNames             7907
AuthorOrders            7907
AuthorPapers            7907
FoSNames                7907
FoSPaperCounts          7907
FoSCiteCounts           7907
DisplayName             5586
NormalizedName          5586
PaperId                 7907
Rank                    7907
Doi                     4961
DocType                 4393
JournalId               5586
ConferenceInstanceId     109
OriginalTitle           7907
Year                    7907
Date                    7907
Publisher               5675
ReferenceCount          7907
CitationCount           7907
EstimatedCitation       7907
URLs                    7286
IndexedAbstract         3642
dtype: int64

### Remove duplicate records from both data sets. 

In [15]:
openSet = set(open.PaperId)
repSet = set(rep.PaperId)
overlap = openSet.intersection(repSet)
cover = openSet.union(repSet)
len(cover)
open = open[~open["PaperId"].isin(overlap)]
rep = rep[~rep["PaperId"].isin(overlap)]
#rep.count()

### Finally we sort the AuthorIds, AuthorNames according to AuthorOrders

In [9]:
def Strip(x):
  return x.strip()

def SortAuthors(row):
  ids = list(map(Strip, row['AuthorIds'].split(";")))
  names = list(map(Strip, row['AuthorNames'].split(";")))
  orders = list(map(Strip, row['AuthorOrders'].split(";")))
  
  Tup = list()
  for i in range(0,len(ids)):
    Id, name, order  = ids[i], names[i], orders[i]
    Tup.append((Id, name, order))
  
  Tup = list(set(Tup))
  Tup.sort(key=lambda x:x[2], reverse=False)
  
  sortedIDs = ""
  flag = 1
  for Id, name, order in Tup:
    if flag == 1:
      sortedIDs += Id
      flag = 0
    else:
      sortedIDs += "; "
      sortedIDs += Id
        
  sortedNames = ""
  flag = 1
  for Id, name, order in Tup:
    if flag == 1:
      sortedNames += name
      flag = 0
    else:
      sortedNames += "; "
      sortedNames += name
  
  return pd.Series([sortedIDs,sortedNames])
  pass

In [10]:
sorted = open.apply(SortAuthors, axis=1)
open['AuthorIdsOrder'] = sorted[0]
open['AuthorNamesOrder'] = sorted[1]

sorted = rep.apply(SortAuthors, axis=1)
rep['AuthorIdsOrder'] = sorted[0]
rep['AuthorNamesOrder'] = sorted[1]
#open.count()

AuthorIds               3499
AuthorNames             3499
AuthorOrders            3499
AuthorPapers            3499
FoSNames                3499
FoSPaperCounts          3499
FoSCiteCounts           3499
DisplayName             1977
NormalizedName          1977
PaperId                 3499
Rank                    3499
Doi                     2375
DocType                 1049
JournalId               1977
ConferenceInstanceId      64
OriginalTitle           3499
Year                    3499
Date                    3499
Publisher               1473
ReferenceCount          3499
CitationCount           3499
EstimatedCitation       3499
URLs                    3335
IndexedAbstract         1452
AuthorIdsOrder          3499
AuthorNamesOrder        3499
dtype: int64

In [16]:
open.to_csv('Open.csv',index=False, encoding='utf-8')
rep.to_csv('Reproduce.csv',index=False, encoding='utf-8')

### check if the author count remains the same after the re-ordering.

In [17]:
import pandas as pd
import numpy as np

openTest = pd.read_csv('Open.csv')
compare1 = openTest.AuthorIdsOrder.str.count("; ")
compare2 = openTest.AuthorIds.str.count("; ")
result = pd.concat([compare1,compare2],axis=1).astype(np.float16)
result['AuthorIdsOrder'].equals(result['AuthorIds'])

True

In [18]:
openTest = pd.read_csv('Reproduce.csv')
compare1 = openTest.AuthorIdsOrder.str.count("; ")
compare2 = openTest.AuthorIds.str.count("; ")
result = pd.concat([compare1,compare2],axis=1).astype(np.float16)
result['AuthorIdsOrder'].equals(result['AuthorIds'])

True